In [21]:
from PIL import Image, ImageOps
import pytesseract

# Open the image and apply EXIF orientation if necessary
image = Image.open("/Users/aaryas127/Documents/GitHub/pharmapp/pharmapp-backend/uploads/IMG_8512.jpeg")
image = ImageOps.exif_transpose(image)

# Extract all text from the image
full_text = pytesseract.image_to_string(image)

# Find the position of "Dr." in the text
start_index = full_text.find("Dr.")

# Check if "Dr." was found, and if so, extract text from that position onward
if start_index != -1:
    text = full_text[start_index:]
    print("Extracted Text Starting from 'Dr.':\n", text)
else:
    print("The keyword 'Dr.' was not found in the image.")



Extracted Text Starting from 'Dr.':
 Dr. Germin Attia MD CCFP
Terry Fox Medical

5380 Terry Fox Way #10
Mississauga Ontario L5V OAS
Pract. No.:139270

Tel: 905-858-3030
Fax: 905-858-3031

Written Date: November 5, 2024
AARYA G SHAH DOB:Jan 27, 2003
44 BRETON AVE

MISSISSAUGA, ON L4Z4K2
905-281-2404

Health Ins.#4741252474

VENTOLIN HFA 100@G
1-2 puffs Q6 Hrs PRN
Qty:1 Repeats:0

Ce ee enn celia

—

Signature:

Requesting: Dr. Germin Attia MD CCFP (139270)
MRP: (81264)

Created by: OSCAR The open-source EMR www.oscarcanada.org



In [22]:
pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install --upgrade transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 23.9 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.2
    Uninstalling transformers-4.45.2:
      Successfully uninstalled transformers-4.45.2
Note: you may need to restart the kernel to use updated packages.


In [30]:
flan_pipeline = pipeline("text2text-generation", model="google/flan-t5-large", device=-1)

In [32]:
from transformers import pipeline

# Initialize the pipeline without device first to troubleshoot
flan_pipeline = pipeline("text2text-generation", model="google/flan-t5-large", device=-1)

# Define a structured prompt for clear formatting
text_input = f"""
Please extract and format the following medical prescription details into distinct sections: 
- **Doctor's Information** (Name, Clinic, Address, Practitioner Number, Contact Info)
- **Patient Details** (Name, DOB, Address, Contact Info, Health Insurance Number)
- **Prescription** (Medication, Dosage Instructions, Quantity, Repeats)
- **Document Metadata** (Written Date, Created By).

Medical Text:
\"\"\"{text}\"\"\"

Return output as structured sections.
"""

# Generate the structured response as a list input
response = flan_pipeline([text_input], max_length=512, do_sample=False)

# Print the structured output
print("Structured Output:\n", response[0]['generated_text'])


Structured Output:
 """Dr. Germin Attia MD CCFP Terry Fox Medical 5380 Terry Fox Way #10 Mississauga Ontario L5V OAS Pract. No.:139270 Tel: 905-858-3030 Fax: 905-858-3031 Written Date: November 5, 2024 AARYA G SHAH DOB:Jan 27, 2003 44 BRETON AVE MISSISSAUGA, ON L4Z4K2 905-281-2404 Health Ins.#4741252474 VENTOLIN HFA 100@G 1-2 puffs Q6 Hrs PRN Qty:1 Repeats:0 Ce ee enn celia — Signature: Requesting: Dr. Germin Attia MD CCFP (139270) MRP: (81264) Created by: OSCAR The open-source EMR www.oscarcanada.org """
